In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import datetime


# 超参数设置
num_epochs = 10
batch_size = 100
learning_rate = 1e-2

# 图像预处理模块
# 先padding, 再翻转，然后裁剪。 数据增广的手段
data_transform = transforms.Compose([
                                    transforms.Pad(4),  # 缩放
                                    transforms.RandomHorizontalFlip(), # 翻转
                                    transforms.RandomCrop(32), # 随机剪裁
                                    transforms.ToTensor()])

# 训练集、测试集
train_dataset = datasets.CIFAR10(root='.\data\cifar10', train=False, 
                              transform=data_transform, download=True)
test_dataset = datasets.CIFAR10(root='.\data\cifar10', train=False,
                            transform=data_transform)

# 数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


# 卷积层

In [11]:
def conv(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=True)

# 残差块（Residual block）

In [12]:
# 普通残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        
        super(ResidualBlock, self).__init__()
        self.conv1 = conv(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = conv(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# 残差网络搭建

In [14]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        """
        主网络
        block: 给定残差块 basic/bottleneck
        layers: 每层残差块数量 list类型
        num_classes: 输出分类数
        
        """
        super(ResNet, self).__init__()
        # 第一个残差块，输入通道数
        self.in_channels = 16
        self.conv = conv(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        #  nn.AvgPool2d需要添加参数ceil_mode=False，否则该模块无法导出为onnx格式
        self.avg_pool = nn.AvgPool2d(8, ceil_mode=False)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        """
        block:
        out_channels:
        blocks: 当前层残差块数目
        stride:
        """
        downsample = None
        """
        当stride不为1，特征图尺寸发生变化，identity需要下采样
        当残差块输入输出通道不一样时，idengtity需要通过1×1卷积改变通道数
        """
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        
        layers = []
        # 残差直接映射部分 -->  添加第一个残差块
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        # 上一层输出通道数，作为下一层输入通道数
        self.in_channels = out_channels
        # 循环添加剩余残差块
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
            
        return nn.Sequential(*layers) # 序列解包
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# 实例化模型

In [15]:
# 实例化一个残差网络模型
model = ResNet(ResidualBlock, [2, 2, 2, 2])

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 用于更新参数组中的学习率
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        para_group['lr'] = lr


# 训练模型

In [ ]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 后向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((i+1)%100 == 0) and ((epoch+1)%5 == 0):
            print("Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}"
                 .format(epoch+1, num_epochs, total_step, loss.item()))
    
    # 学习率衰减策略
    if (epoch+1)%20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                  
